# CONDUCCIÓN AUTÓNOMA
María Rodríguez Palomo

En este script veremos como podemos crear el código para la conducción autonoma de un vehículo en Carla Simulator.
Para ello haremos que el coche circule por el carril en el que lo creamos. También detectaremos semáforos y señales. A su vez, haremos un cambio de clima en el simulador.

### PARTE 0: IMPORTAMOS LAS LIBRERIAS NECESARIAS

In [1]:
import carla 
import cv2 
import numpy as np 
from matplotlib import pyplot as plt 
import time
import random
import math
import glob
import os
import sys
import argparse
import keyboard as kb
import re

### PARTE 1: CREACIÓN DE LOS AGENTES

#### MUNDO
En nuestro caso vamos a trabajar con el circuito de fuera de la ciudad del mundo 05.

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### VISTA
Vamos a cambiar la perspectiva de vista de carla simulator para poder ver en todo momento que hace el vehículo

In [3]:
loc=carla.Transform(carla.Location(x=171.557922, y=157.525208, z=1.138501), carla.Rotation(pitch=0.533207, yaw=-237.194565, roll=0.006472))
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc.location + carla.Location(x=6,y=-6,z=4),carla.Rotation(yaw=loc.rotation.yaw))
spectator.set_transform(spectator_pos)

#### COCHE
Vamos a crear un coche Tesla color fuccia en la localización especificada en loc.

In [4]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc)

#### SENSOR
Crearemos una cámara que vaya pegada al coche que nos muestre que está pasando en todo momento.

In [5]:
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

In [6]:
CAMERA_POS_Z = 3
CAMERA_POS_X = -5
camera_bp=world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')
camera_init_trans= carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera=world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w= camera_bp.get_attribute('image_size_x').as_int()
image_h= camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

### PARTE 2: CONDUCCIÓN EN EL CARRIL

#### VELOCIDAD
Creamos una función que según la velocidad limite nos calcule la aceleración necesaria para llevar una velocidad lo más parecida posible sin pasarnos

In [7]:
def maintain_speed(s,preferred_speed,speed_threshold):
    if s >= preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGULO
Creamos una función que nos calcule el ángulo entre dos puntos del mapa.

In [8]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

#### STEER
Creamos una función que nos calcule el angulo y la dirección a aplicar al vehículo.

In [9]:
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

### PARTE 3: CAMBIO DEL TIEMPO
Creamos una función que nos cambie el tiempo aleatorimente.

In [10]:
def weather():
    weather = carla.WeatherParameters(
            cloudiness=random.randint(0,100),
            precipitation=random.randint(0,100),
            sun_altitude_angle=random.randint(0,100),
            precipitation_deposits =random.randint(0,100),
            fog_density =random.choice([0.0,0.1,0.3,0.4,0.95]),
            wetness = random.randint(0,100))
    world.set_weather(weather)

### PARTE 4: SEMÁFOROS

#### LUZ
Creamos una función que nos devuelva el color de la luz del semáforo correspondiente a nuestro carril y zona.

In [11]:
def is_traffic_light_red(vehicle, world):
    location = vehicle.get_location()
    actor_list = world.get_actors()

    for actor in actor_list:
        if actor.type_id.startswith("traffic.traffic_light"):
            traffic_light_location = actor.get_location()
            distance = location.distance(traffic_light_location)
            if distance < 37.0:
                traffic_light_state = actor.get_state()
                vehicle_waypoint = world.get_map().get_waypoint(location)
                traffic_light_waypoint = world.get_map().get_waypoint(traffic_light_location)
                if traffic_light_waypoint.lane_id == 1:
                    if traffic_light_state == carla.TrafficLightState.Red:
                        return True
                    elif traffic_light_state == carla.TrafficLightState.Green:
                        return False

#### FRENO
Fnción que frena el vehículo si hay un semáforo en rojo y si no aplica la velocidad previamente calculada.

In [12]:
def freno(light,brake_intensity,vehicle,estimated_throttle,steer_input):
    if light:
        control = carla.VehicleControl(brake=brake_intensity)
        vehicle.apply_control(control)
    else:    
        vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))

### PARTE 6: SEÑALES
Creación de una función que detecta señales de velocidad que afectan al carril del coche y devuelve la velocidad limite.

In [13]:
def signs(vehicle, world):
    location = vehicle.get_location()
    actor_list = world.get_actors()

    for actor in actor_list:
        if actor.type_id.startswith("traffic.speed_limit"):
            traffic_sign_location = actor.get_location()
            distance = location.distance(traffic_sign_location)
            if distance < 10.0:
                # Obtiene el waypoint del vehículo y de la señal de tráfico
                vehicle_waypoint = world.get_map().get_waypoint(location)
                sign_waypoint = world.get_map().get_waypoint(traffic_sign_location)
                # Verifica si la señal de tráfico está en la misma carretera que el vehículo
                if sign_waypoint.lane_id == 4:
                    # Verifica si la señal de tráfico tiene información de velocidad límite
                    match = re.search(r'traffic\.speed_limit\.(\d+)', actor.type_id)
                    numero = match.group(1) 
                    speed_limit_kph = int(numero)
                    return speed_limit_kph
    return 0

### PARTE 7: CONDUCCIÓN
Codigo principal con llamamiento a las funciones para que la conducción autonoma funcione.

In [14]:
#Definir variables 
brake_intensity = 0.8
i=0
preferred_speed = 90
speed_threshold = 2
MAX_STEER_DEGREES = 20
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
quit=False

while True:
    #Cámara
    world.tick()
    if cv2.waitKey(1)==ord('q'): #si pulsamos la tecla q la pestaña desaparece
        quit=True
        break
    image = camera_data['image']
    
    #Cambio de clima
    i +=1
    if i ==50:
        i=0
        weather()
        
    #Localización actual del vehículo
    loc=vehicle.get_location()
    current_w = map.get_waypoint(loc)
    waypoint_separation = 5
    next_w0 = list(current_w.next(waypoint_separation))[0]
    next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    
    #Calculo de velocidad y angulo
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    image=cv2.putText(image,'Speed: '+str(int(speed))+' km/h', (30,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    predicted_angle = get_angle(vehicle,next_w)
    steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
    sign=signs(vehicle, world)
    if sign!=0:
        preferred_speed=sign
    light=is_traffic_light_red(vehicle, world)
    freno(light,brake_intensity,vehicle,estimated_throttle,steer_input)

    #Cámara
    cv2.imshow('RGB Camera', image)
cv2.destroyAllWindows()
camera.stop()

### PARTE 8: DESTRUIR
Destrucción de los agentes utilizados.

In [15]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()